## vLLM 을 활용한 Multi-LoRA

하나의 베이스 모델만 메모리에 로드한 상태에서 여러 개의 LoRA 를 즉시 적용하고 전환할 수 있다.  
이는 자원 사용을 획기적으로 줄이고 모델 전환 속도를 크게 높인다.

In [1]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from huggingface_hub import snapshot_download

model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

llm = LLM(model=model_id, enable_lora=True, max_lora_rank=256)

/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-03 12:20:39,015	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-03 12:20:47 config.py:510] This model supports multiple tasks: {'embed', 'reward', 'score', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 01-03 12:20:47 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='allganize/Llama-3-Alpha-Ko-8B-Instruct', speculative_config=None, tokenizer='allganize/Llama-3-Alpha-Ko-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, serv

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:04<00:13,  4.34s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:09<00:09,  4.66s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:10<00:03,  3.13s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:15<00:00,  3.76s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:15<00:00,  3.81s/it]



INFO 01-03 12:21:08 model_runner.py:1099] Loading model weights took 14.9634 GB
INFO 01-03 12:21:08 punica_selector.py:11] Using PunicaWrapperGPU.
INFO 01-03 12:21:13 worker.py:241] Memory profiling takes 5.49 seconds
INFO 01-03 12:21:13 worker.py:241] the current vLLM instance can use total_gpu_memory (39.39GiB) x gpu_memory_utilization (0.90) = 35.45GiB
INFO 01-03 12:21:13 worker.py:241] model weights take 14.96GiB; non_torch_memory takes 1.51GiB; PyTorch activation peak memory takes 1.24GiB; the rest of the memory reserved for KV Cache is 17.75GiB.
INFO 01-03 12:21:14 gpu_executor.py:76] # GPU blocks: 9085, # CPU blocks: 2048
INFO 01-03 12:21:14 gpu_executor.py:80] Maximum concurrency for 8192 tokens per request: 17.74x
INFO 01-03 12:21:23 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occu

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:23<00:00,  1.50it/s]

INFO 01-03 12:21:47 model_runner.py:1535] Graph capturing finished in 24 secs, took 2.37 GiB
INFO 01-03 12:21:47 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 38.89 seconds


In [2]:
sampling_params_lora1 = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=50)
lora_adapter1 = "daje/chapter5_psychological_chatbots"
lora_adapter1_path = snapshot_download(repo_id=lora_adapter1)
lora1 = LoRARequest("lora1", 1, lora_adapter1_path)

Fetching 8 files: 100%|██████████| 8/8 [00:46<00:00,  5.76s/it]
/tmp/ipykernel_1140750/1501373901.py:4: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora1 = LoRARequest("lora1", 1, lora_adapter1_path)


In [3]:
sampling_params_lora2 = SamplingParams(temperature=0.1, max_tokens=50)
lora_adapter2 = "daje/chapter5_code-llama3-8B-text-to-sql-ver0.1"
lora_adapter2_path = snapshot_download(repo_id=lora_adapter2)
lora2 = LoRARequest("lora2", 2, lora_adapter2_path)

Fetching 7 files: 100%|██████████| 7/7 [00:33<00:00,  4.72s/it]
/tmp/ipykernel_1140750/1020445224.py:4: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora2 = LoRARequest("lora2", 2, lora_adapter2_path)


## 테스트

In [4]:
prompts_lora1 = [
    "일요일인데 새벽6시에 일어났어 ㅜㅜ",
    "요즘 대상포진이 걸려서 고생했어",
]

outputs = llm.generate(prompts_lora1, sampling_params_lora1, lora_request=lora1)

for output in outputs:
    generated_text = output.outputs[0].text
    print(generated_text)
    print('------')

Processed prompts: 100%|██████████| 2/2 [00:53<00:00, 26.65s/it, est. speed input: 0.54 toks/s, output: 1.88 toks/s]

. 평소에는 매일 일어나는 것이 힘들었는데 요즘은 정말 힘들어요. 아침에 일어나기 싫은 마음이 너무 커서 밤에도 잠을 잘 못자고, 주말이 되면 정말 기
------
. 이제는 매일 밤마다 눈이 따가워지고, 눈 앞이 흐려지고, 눈이 불편해져서 일도 제대로 못하고, 집에서도 편하게 쉬지도 못하고, 잠도 못자고,
------


In [5]:
prompts_lora2 = [
    """Task:최고 총액을 말해줘.'
SQL table: CREATE TABLE table_12014 (
    "Rider" text,
    "Horse" text,
    "Faults" text,
    "Round 1 + 2A Points" text,
    "Total" real
)
SQL query:""",
    "sql로 평균 구하는거 알려줘.",
]

outputs = llm.generate(prompts_lora2, sampling_params_lora2, lora_request=lora2)

for output in outputs:
    generated_text = output.outputs[0].text
    print(generated_text)
    print('------')

Processed prompts: 100%|██████████| 2/2 [00:27<00:00, 13.52s/it, est. speed input: 2.85 toks/s, output: 2.44 toks/s]

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

SELECT MAX("Total") FROM table_12014
------
 그리고 이름을 내림차순으로 정렬해줘.
SQL table: CREATE TABLE table_203_203 (
    id number,
    "rank" number,
    "name" text,
    "nationality" text,
    "
------
